# low-rank Krylov Subspace Approximation (KSA) for density matrix SCF optimization

In [1]:
import sys
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/") # path to PySEQM
import seqm
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
from seqm.seqm_functions.read_xyz import read_xyz

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

DTYPE = torch.float64
torch.set_default_dtype(DTYPE)

seqm.seqm_functions.scf_loop.debug = True # print SCF steps
seqm.seqm_functions.scf_loop.MAX_ITER=1000 # max SCF iterations
seqm.seqm_functions.scf_loop.CANON_DM_PRT_ITER = 8 # number of iterations in DM perturbation calcs


/home/maxim/anaconda3/envs/hipnn_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%time
files = [
        'C380.xyz',
        #'coronene.xyz',
        #'GramicidinS/GS_dp.xyz',
        #'Fullerenes/C380.xyz'
        ]

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device, dtype=DTYPE)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))

# parameters dict for KSA-XL-BOMD needs to be passed to 'scf_converger'
xl_bomd_params={'k':6, 'max_rank':3, 'err_threshold':0.0, 'T_el':1500}

seqm_parameters = {
                   'method' : 'PM6_SP',  # AM1, MNDO, PM3, PM6, PM6_SP. PM6_SP is PM6 without d-orbitals. Effectively, PM6 for the first two rows of periodic table
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [3,xl_bomd_params], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                                         # [3, xl_bomd_params], low-rank KSA scf optimization
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

charges = torch.tensor([2], device = device)
molecules = Molecule(const, seqm_parameters, coordinates, species, charges = charges).to(device)

### Create electronic structure driver:
esdriver = Electronic_Structure(seqm_parameters).to(device)

### Run esdriver on molecules:
esdriver(molecules)


scf KSA step     :   1 | MAX ΔE[   0]: 1792498.2235411 | MAX ΔDM[   0]:   38.8795500  | N not converged: 1
scf KSA step     :   2 | MAX ΔE[   0]:  124.5301600 | MAX ΔDM[   0]:    2.8769164  | N not converged: 1
scf KSA step     :   3 | MAX ΔE[   0]:    0.2128476 | MAX ΔDM[   0]:    0.9973304  | N not converged: 1
scf KSA step     :   4 | MAX ΔE[   0]:    0.2064682 | MAX ΔDM[   0]:    0.5803997  | N not converged: 1
scf KSA step     :   5 | MAX ΔE[   0]:    0.2946657 | MAX ΔDM[   0]:    0.3121098  | N not converged: 1
scf KSA step     :   6 | MAX ΔE[   0]:    0.2240135 | MAX ΔDM[   0]:    0.2848897  | N not converged: 1
scf KSA step     :   7 | MAX ΔE[   0]:    0.1638374 | MAX ΔDM[   0]:    0.2583678  | N not converged: 1
scf KSA step     :   8 | MAX ΔE[   0]:    0.1126863 | MAX ΔDM[   0]:    0.2190037  | N not converged: 1
scf KSA step     :   9 | MAX ΔE[   0]:    0.0744728 | MAX ΔDM[   0]:    0.1766672  | N not converged: 1
scf KSA step     :  10 | MAX ΔE[   0]:    0.0492446 | MAX ΔDM

In [4]:
print(' Total Energy (eV):\n', molecules.Etot)
print('\n Electronic Energy (eV): ', molecules.Eelec)
print('\n Nuclear Energy (eV):\n', molecules.Enuc)
print('\n Heat of Formation (ev):\n', molecules.Hf)
print('\n HOMO-LUMO gap (eV):\n', molecules.e_gap)
print('\n Forces (eV/A):\n', molecules.force)

 Total Energy (eV):
 tensor([-46467.1159], device='cuda:0')

 Electronic Energy (eV):  tensor([-1792623.9752], device='cuda:0')

 Nuclear Energy (eV):
 tensor([1746156.8593], device='cuda:0')

 Heat of Formation (ev):
 tensor([125.4200], device='cuda:0')

 HOMO-LUMO gap (eV):
 tensor([2.8197], device='cuda:0')

 Forces (eV/A):
 tensor([[[-0.0995,  0.2642,  0.0768],
         [-0.1440, -0.4239,  0.1285],
         [ 0.3575,  0.2699, -0.0444],
         ...,
         [ 0.3187,  0.9286,  0.5780],
         [ 0.5077, -0.8989, -0.6737],
         [ 1.3635,  0.3599,  0.1332]]], device='cuda:0')
